In [ ]:
!pip install gradio

In [ ]:
!pip install pypdf


In [ ]:
!pip install chromadb


In [ ]:
!pip install google.generativeai


In [ ]:
!pip install langchain-google-genai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.0/372.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.4
    Uninstalling google-ai-generativelanguage-0.6.4:
      Successfully uninstalled google-ai-generativelanguage-0.6.4
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.5.4
    Uninstalling google-generativeai-0.5.4:
      Successfully uninstalled google-generativeai-0.5.4


In [ ]:
!pip install langchain


  Using cached langchain-0.2.9-py3-none-any.whl (987 kB)
  Using cached langchain_text_splitters-0.2.2-py3-none-any.whl (25 kB)


In [ ]:
!pip install langchain_community


  Using cached langchain_community-0.2.7-py3-none-any.whl (2.2 MB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached marshmallow-3.21.3-py3-none-any.whl (49 kB)


In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.9 MB/s eta 0:00:00


In [ ]:
from google.colab import files

# Prompt the user to upload a file
uploaded = files.upload()


Saving grad-student-handbook.pdf to grad-student-handbook (1).pdf


In [ ]:
#Import Python modules
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import Chroma
from gradio import Interface
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter


GOOGLE_API_KEY = ""

with open("api.txt") as f:
    GOOGLE_API_KEY = f.read().strip()
    f.close()


#Load the model
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)


#Load the PDF and extract text

def get_pdf_text(pdf_doc):
    text=""
    pdf_reader= PdfReader(pdf_doc)
    for page in pdf_reader.pages:
        text+= page.extract_text()
    return  text

# create chunks using RecursiveCharacterTextSplitter

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

# create a vector store using Chroma
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)
    vector_store = Chroma.from_texts(text_chunks,embeddings)
    return vector_store


pdf_text = get_pdf_text("grad-student-handbook.pdf")
text_chunks = get_text_chunks(pdf_text)
vectordb = get_vector_store(text_chunks)

#Configure Chroma as a retriever with top_k=5
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

#Create the prompt template
template = """
Provide a response to the question based on the context provided.
If an answer is not in the context, say 'I don't know the answer'.
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)


chat_history = []

# Function to handle chat
def chat_response(question):
    response=retrieval_chain.invoke({"input":question})
    chat_history.append(("User", question))
    chat_history.append(("Bot", response["answer"]))
    return chat_history


# Create the chatbot interface
interface = Interface(fn=chat_response, inputs="text", outputs="text",
                      title="Q&A Chatbot with Gemini")

# Run the Gradio interface
interface.launch()





Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2596ae8b96e927fd1d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
#Import Python modules
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import Chroma
import gradio as gr

GOOGLE_API_KEY = ""

with open("api.txt") as f:
    GOOGLE_API_KEY = f.read().strip()
    f.close()


#Load the models
llm = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOOGLE_API_KEY)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)

#Load the PDF and create chunks
loader = PyPDFLoader("grad-student-handbook.pdf")
text_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)
pages = loader.load_and_split(text_splitter)

#Turn the chunks into embeddings and store them in Chroma
vectordb=Chroma.from_documents(pages,embeddings)

#Configure Chroma as a retriever with top_k=5
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

#Create the prompt template
template = """
Provide a response to the question based on the context provided.
If an answer is not in the context, say 'I don't know the answer'.
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)



# Function to handle chat
def chat_response(question, chat_history):
    response=retrieval_chain.invoke({"input":question})
    #chat_history.append([question, response["answer"]])
    return response["answer"]


# Create the chatbot interface
gr.ChatInterface(chat_response).launch()




Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4898e5944699e048e1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
